Dieses Notebook ist angelehnt an das Buch *Python Data Science Handbook* von Jake VanderPlas, auch verfügbar auf [GitHubPages](https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html#Example:-Predicting-Bicycle-Traffic).

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

c:\Users\Cris\anaconda3\lib\site-packages\scipy\__init__.py:173: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Praktikum: Session 5
[Video](https://mstream.hm.edu/paella/ui/watch.html?id=bcb4dac3-6114-475e-a5ef-436926cf0497)

In dieser Session stehen zwei Punkte im Vordergrund:

1.   Datenaufbereitung
2.   Interpretation von Modellen

Um den zweiten Punkt gut abbilden zu können, verwenden wir ein lineares Modell.

Als Anwendungsfall betrachten wir die Anzahl der Fahrradfahrten über die Fremont Bridge in Seattle (dort sind entsprechende Sensoren installiert und die Daten seit Oktober 2012 verfügbar). Ziel ist es, diese Anzahl der Fahrten für einen gegebenen Tag vorherzusagen. Zu diesem Zweck wollen wir ein lineares Modell erstellen.

## 1. (Roh-)Daten
Die Rohdaten der Fremont Bridge können einfach heruntergeladen werden:

In [2]:
!curl -o FremontBridge.csv https://data.seattle.gov/api/views/65db-xm6k/rows.csv?accessType=DOWNLOAD

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  319k    0  319k    0     0   172k      0 --:--:--  0:00:01 --:--:--  172k
100 1742k    0 1742k    0     0   598k      0 --:--:--  0:00:02 --:--:--  598k
100 2975k    0 2975k    0     0   803k      0 --:--:--  0:00:03 --:--:--  803k


In [3]:
import pandas as pd
counts = pd.read_csv('FremontBridge.csv', index_col='Date', parse_dates=True)

In [4]:
counts.sort_index()

,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk"
Date,,,
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 02:00:00,2.0,1.0,1.0
2012-10-03 03:00:00,5.0,2.0,3.0
2012-10-03 04:00:00,7.0,6.0,1.0
...,...,...,...
2023-08-31 19:00:00,224.0,72.0,152.0
2023-08-31 20:00:00,142.0,59.0,83.0
2023-08-31 21:00:00,67.0,35.0,32.0


### 0. Säuberung der Daten
#### 0.1 Doppelte Daten
Zu allererst überprüfen wir, ob der Datensatz doppelte Daten enthält:

In [5]:
print("counts enthält %d Einträge." % counts.index.shape)
print("Es gibt %d einzigartige Zeitstempel." % counts.index.unique().shape)

counts enthält 95640 Einträge.
Es gibt 95640 einzigartige Zeitstempel.


Hier scheint also alles in Ordnung zu sein. In einem früher verfügbaren Datensatz (Stand 11.11.2020) waren sehr viele Daten doppelt vorhanden. Folgender optionale Unterabschnitt zeigt, wie mit einer solchen Problematik umgegangen werden kann: 

#### 0.2 Bereinigung doppelter Daten
Um überhaupt das Problem doppelt vorhandener Daten zu haben, laden wir den Datensatz in der Version vom 11.11.2020, welcher [hier](datasets/FremontBridge2020.csv) verfügbar ist (wenn Sie Colab verwenden, laden Sie den Datensatz von GitLab runter und dann in Colab wieder hoch).

In [6]:
counts2020 = pd.read_csv('datasets/FremontBridge2020.csv', index_col='Date', parse_dates=True)

In [7]:
counts2020.sort_index()

,Fremont Bridge Total,Fremont Bridge East Sidewalk,Fremont Bridge West Sidewalk
Date,,,
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 00:00:00,13.0,4.0,9.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 01:00:00,10.0,4.0,6.0
2012-10-03 02:00:00,2.0,1.0,1.0
...,...,...,...
2020-09-30 19:00:00,156.0,51.0,105.0
2020-09-30 20:00:00,70.0,27.0,43.0
2020-09-30 21:00:00,40.0,17.0,23.0


In [8]:
print("counts2020 enthält %d Einträge." % counts2020.index.shape)
print("Es gibt aber nur %d einzigartige Zeitstempel." % counts2020.index.unique().shape)
print("Hier ein Auszug aus der Häufigkeitsverteilung:")
counts2020.index.value_counts()

counts2020 enthält 136334 Einträge.
Es gibt aber nur 70080 einzigartige Zeitstempel.
Hier ein Auszug aus der Häufigkeitsverteilung:


2019-06-15 03:00:00    2
2016-02-18 09:00:00    2
2018-03-21 17:00:00    2
2013-08-26 17:00:00    2
2019-05-21 16:00:00    2
                      ..
2020-07-10 08:00:00    1
2020-06-08 13:00:00    1
2020-09-12 21:00:00    1
2020-03-30 14:00:00    1
2020-03-29 15:00:00    1
Name: Date, Length: 70080, dtype: int64

Bevor wir diese entfernen, sollten wir prüfen, ob es fehlende Daten gibt. Diese sollten ggf. entfernt werden, **bevor** doppelte Zeilen gelöscht werden (*warum?*).

In [9]:
counts2020[counts2020.isna().any(axis=1)]

,Fremont Bridge Total,Fremont Bridge East Sidewalk,Fremont Bridge West Sidewalk
Date,,,
2013-06-14 09:00:00,NaN,NaN,NaN
2013-06-14 10:00:00,NaN,NaN,NaN
2014-03-09 02:00:00,NaN,NaN,NaN
2015-03-08 02:00:00,NaN,NaN,NaN
2015-04-21 11:00:00,NaN,NaN,NaN
2015-04-21 12:00:00,NaN,NaN,NaN
2016-03-13 02:00:00,NaN,NaN,NaN
2017-03-12 02:00:00,NaN,NaN,NaN
2018-03-11 02:00:00,NaN,NaN,NaN


Es gibt also 21 Zeilen, die fehlende Daten enthalten. Da wir wissen, dass es Doppelungen in den Indices gibt, besteht die Chance, dass die hier fehlenden Daten in der gedoppelten Zeile eingetragen sind. Prüfen wir das:

In [10]:
counts2020.loc[counts2020[counts2020.isna().any(axis=1)].index]

,Fremont Bridge Total,Fremont Bridge East Sidewalk,Fremont Bridge West Sidewalk
Date,,,
2013-06-14 09:00:00,NaN,NaN,NaN
2013-06-14 09:00:00,NaN,NaN,NaN
2013-06-14 10:00:00,NaN,NaN,NaN
2013-06-14 10:00:00,NaN,NaN,NaN
2014-03-09 02:00:00,NaN,NaN,NaN
2014-03-09 02:00:00,NaN,NaN,NaN
2015-03-08 02:00:00,NaN,NaN,NaN
2015-03-08 02:00:00,NaN,NaN,NaN
2015-04-21 11:00:00,NaN,NaN,NaN


Eine Korrektur rentiert sich hier wohl eher nicht... Daher löschen wir sie:

In [11]:
counts2020.dropna(inplace=True)

Nun wollen wir diejenigen Zeilen, der Index (also der Zeitstempel) doppelt auftreten löschen; genauer: wir wollen die Doppelungen löschen, die jeweils erste Instanz soll beibehalten werden.

Mit ``counts.index`` können wir auf den Index zugreifen. Die Methode ``duplicated()`` liefert uns Informationen darüber, ab es sich jeweils um ein Duplikat handelt (``True``) oder nicht (``False``). Die jeweils erste Instanz wird dabei nicht als Duplikat markiert. Durch den Operator ``~`` können wir eine logische Negation durchführen und erhalten damit einen Index, der genau dort den Wert ``True`` hat, wo *kein* Duplikat steht. Die dazu korrespondierenden Zeilen wollen wir behalten.

In [12]:
~counts2020.index.duplicated()

array([ True,  True,  True, ...,  True,  True,  True])

In [13]:
counts2020 = counts2020[~counts2020.index.duplicated()]

In [14]:
counts2020.index.shape

(70070,)

Somit wären auch Daten mit Doppelungen aufbereitet, der Einschub endet hier und wir gehen zurück zu den aktuellen Daten.

#### 0.3 Aufbereitung der Daten
Auch bei den aktuellen Daten überprüfen wir, ob Samples mit fehlenden Einträgen vorhanden sind:

In [15]:
counts[counts.isna().any(axis=1)]

,"Fremont Bridge Sidewalks, south of N 34th St","Fremont Bridge Sidewalks, south of N 34th St Cyclist East Sidewalk","Fremont Bridge Sidewalks, south of N 34th St Cyclist West Sidewalk"
Date,,,
2013-03-10 02:00:00,NaN,NaN,NaN
2013-06-14 09:00:00,NaN,NaN,NaN
2013-06-14 10:00:00,NaN,NaN,NaN
2014-03-09 02:00:00,NaN,NaN,NaN
2015-03-08 02:00:00,NaN,NaN,NaN
2015-04-21 11:00:00,NaN,NaN,NaN
2015-04-21 12:00:00,NaN,NaN,NaN
2016-03-13 02:00:00,NaN,NaN,NaN
2017-03-12 02:00:00,NaN,NaN,NaN


Diese 26 Zeilen löschen wir:

In [16]:
counts.dropna(inplace=True)

Offenbar sind hier die Überquerungen in einem Stundenraster vorhanden. Außerdem ist neben dem Gesamtverkehr auch noch die Aufteilung in östliche und westliche Sput vorhanden (westlich: Richtung Downtown). Wir wollen diese Daten noch abändern, denn:
- Wir sind nur am gesamten Verkehr interessiert.
- Wir interessieren uns nur für die gesamten Überquerungen eines ganzen Tages.

Für diese für uns relevanten Informationen legen wir einen DataFrame ``daily`` an:

In [19]:
daily = counts.resample('d').sum()
daily['Total'] = daily['Fremont Bridge Sidewalks, south of N 34th St']
daily = daily[['Total']]

In [20]:
daily.head()

,Total
Date,
2012-10-03,3521.0
2012-10-04,3475.0
2012-10-05,3148.0
2012-10-06,2006.0
2012-10-07,2142.0


Basierend darauf können wir noch kein Modell erstellen, wir werden weitere Daten brauchen.
- Überlegen Sie, von welchen Einflussgrößen der tägliche Fahrradverkehr abhängen könnte.

Folgende Daten wollen wir als Features verwenden:
1. Wochentag
2. Feiertag ja/nein
3. Zeitpunkt im Jahr (genauer: Stunden mit Tageslicht)
4. Temperatur
5. Niederschlag
6. Trend von Jahr zu Jahr

### 1. Wochentag
Hierfür brauchen wir keine neuen Daten, denn die Information ist implizit mit dem Datum bereits vorhanden. Allerdings müssen wir sie extrahieren, denn sonst kann sie ein lineares Modell nicht bekommen (da nichtlinearer, periodischer Zusammenhang).
Wir wollen für jeden Wochentag eine eigene Spalte anlegen, die genau dann eine 1 enthält, wenn der aktuelle Tag dieser Wochentag ist, sonst soll sie 0 enthalten.

* Hinweis: ``daily.index`` gibt nur den Index des DataFrame ``daily`` zurück. In diesem Fall besteht dieser aus kalendarischen Daten, welche direkt in Form eine ``DatetimeIndex`` vorliegen. Das ist praktisch, denn solch ein ``DatetimeIndex`` hat die Methode ``dayofweek``, d.h. ``daily.index.dayofweek`` gibt die Information über den Wochentag zurück.

### 2. Feiertage
Diese Information ist rein aus dem Datum noch nicht ablesbar, wir brauchen zusätzlich noch einen Feiertagskalender. Dieser wird im Folgenden eingelesen und unserem DataFrame ``daily`` hinzugefügt.
- Untersuchen Sie, wie dies geschieht.

In [45]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
holidays = cal.holidays('2012', 'today')
daily = daily.join(pd.Series(1, index=holidays, name='holiday'))
#daily['holiday'].fillna(0, inplace=True)

### 3. Tageslichtstunden
Auch die Information, wie lang die einzelnen Tage sind (Tageslichtstunden), ist in den Daten noch nicht vorhanden. Diese kann aber aus dem Datum und der geographischen Lage *berechnet* werden. In der unten stehenden Funktion ``hours_of_daylight`` ist die astronomische Standardberechnung implementiert. Nutzen Sie diese, um dem DataFrame ``daily`` eine weitere Spalte ``daylight_hrs`` hinzuzufügen, welche diese Information enthält. Stellen Sie diese neue Spalte graphisch dar, indem Sie ``daily['daylight_hrs'].plot()`` verwenden.

*Hinweis:* Mit dem Befehl ``map(hours_of_daylight, Daten)`` können Sie die Funktion ``hours_of_daylight`` auf Daten anwenden, die Funktion ``list(...)`` macht daraus eine Liste.

In [47]:
from datetime import datetime

def hours_of_daylight(date, axis=23.44, latitude=47.61):
    """Compute the hours of daylight for the given date"""
    days = (date - datetime(2000, 12, 21)).days
    m = (1. - np.tan(np.radians(latitude))
         * np.tan(np.radians(axis) * np.cos(days * 2 * np.pi / 365.25)))
    return 24. * np.degrees(np.arccos(1 - np.clip(m, 0, 2))) / 180.

### 4. Temperatur / 5. Niederschlag
Diese Informationen sind ganz offensichtlich nicht in den Daten vorhanden und können daraus auch nicht berechnet werden. Wir müssen daher eine zweite Datenquelle verwenden. Wir verwenden die Wetterdaten der [NOAA](https://www.ncdc.noaa.gov/cdo-web/search?datasetid=GHCND). Damit Sie die Daten dort nicht einzeln bestellen müssen, stelle ich diese [hier](datasets\BicycleWeather2023.csv) bereit. Importen Sie diese. 

*Hinweis:* Da wir die täglichen Wetterdaten unserer ``daily`` Tabelle hinzufügen wollen, wäre es wohl sinnvoll, wenn die Datumsinformationen aus der csv-Datei direkt aufbereitet würden (eben als Datum) und nicht als String (also als inhaltsleerer Text) vorliegen würden. ``pd.read_csv`` hat entsprechende Optionen.

*Hinweis:* Gehen Sie analog zum Einlesen der Datei *FremontBridge.csv*, siehe oben, vor.

Bei der Durchschittstemperatur gibt es einige fehlende Daten (wie könnte man das herausfinden?). Legen Sie eine neue Spalte ``weather['Temp (C)']`` an, welche falls vorhanden die Durchschnittstemperatur ``TAVG`` und ansonsten als Näherung hierfür den Mittelwert aus ``TMIN`` und ``TMAX``.

*Hinweis:* Mit ``weather.loc[Zeilen, Spalten]`` kann man (auch schreibend) auf die durch Zeilen und Spalten spezifizierten Einträge zugreifen. ``weather['Temp (C)'].isnull()`` kann für die Zeilenwahl hilfreich sein.

Was den Niederschlag angeht, so wäre es evtl. auch relevant explizit zu wissen, ob es an einem Tag regnet (also ``PRCP!=0``) oder eben nicht (also ``PRCP==0``). Legen Sie hierfür eine neue Spalte ``weather[dry day]`` an, die an trockenen Tagen 1 enthält und sonst 0.

*Hinweis:* Ein Vergleich wie ``PRCP==0`` liefert einen Wahrheitswert, d.h. ``True`` oder ``False`` zurück. Damit kann unser Modell aber nicht rechnen. Ein nachgestelltes ``.astype(int)`` sorgt dafür, dass ``True`` als 1 und ``False`` als 0 angegeben wird.

Diese Wetterdaten (also ``'PRCP', 'Temp (C)', 'dry day'``) sollen nun den ``daily`` Daten hinzugefügt werden. Hierfür gibt es die Methode ``join``: Durch den Befehl ``df1.join(df2)`` wird ein neuer DataFrame erstellt, welcher die Daten aus ``df1`` und die Daten aus ``df2`` zusammengefügt enthält. Dabei sollten die Indices der beiden DataFrames übereinstimmen.

### 6. Trend von Jahr zu Jahr
Um einen etwa vorhandenen langfristigen Trend ggf. mit aufnehmen zu können, soll eine weiteres Feature angeben, wie viele Tage seit Beginn der Messung vergangen sind. Damit haben wir eine monoton steigende Größe.

Legen Sie ein Feature ``annual`` an, das die beschriebene Größe skaliert mit 1/365 angibt. D.h. ``annual`` soll nach einem Jahr den Wert 1, nach zwei Jahren den Wert 2  usw. haben.

*Hinweis:* Das Datum ist im Index gespeichert, auf den mit ``daily.index`` zugegriffen werden kann. ``daily.index[0]`` gibt das erste Datum als ``Timestamp`` zurück. Mit solchen Timestamps kann ganz einfach gerechnet werden, so gibt etwa die Differenz zweier Timestamps den dazwischen liegenden Zeitraum an (als ``Timedelta``-Objekt, mit ``.days`` extrahiert man daraus die Tage).

### 7. Pandemie
Man kann davon ausgehen, dass die Pandemie-Situation u.a. den Fahrradverkehr über die Fremont-Bridge signifikant beeinflusst; dies bestätigt sich auch in den Daten.



1.   Formulieren Sie eine Hypothese (oder zwei...) bzgl. der Auswirkung
2.   Fügen Sie dem DataFrame ``daily`` eine neue Spalte ``pandemic`` hinzu, die für alle Daten ab dem 01.03.2020 den Wert 1 enthält, sonst 0.
3.   Untersuchen Sie die Daten mit dem Auge und entscheiden Sie, ob der Pandemie-Flag ab einem gewissen Zeitpunkt wieder entfernt werden sollte. Ab wann?

Wir prüfen, ob nun noch irgendwo Werte fehlen (schauen aber vielleicht nicht so genau hin, hätte ja auch sein können, dass wir es vergessen haben... werden wir später schon noch merken):

## 2. Modell
Entnehmen Sie aus ``daily`` die Features Matrix ``X`` und den Labels Vektor ``y``.

Trainieren Sie ein ``LinearRegression`` Modell auf die Daten. Setzen Sie den Hyperparameter ``fit_intercept=False``. Plotten Sie die erhaltenen Ergebnisse und vergleichen Sie diese (im Plot) mit den echten Daten.

*Hinweis:* Sie können (eine Auswahl) eines DataFrames ``df`` einfach plotten durch ``df[['Item_1', ..., 'Item_N']].plot(alpha=0.5)``. Dabei sorgt ``alpha=0.5`` für einen nicht deckenden Plot, so dass sich ggf. überlagernde Linien sichtbar bleiben.

*Hinweis:* Offenbar entsteht hier ein Fehler. Finden Sie raus, was hier falsch läuft.

*Tipp/Spoiler:* Wenn Sie die Daten nur bis November 2021 betrachten, ist alles gut...

Schauen wir die ``X`` Daten für Dezember 2021 doch mal an:

In [125]:
start = '2021-12-10'
end = '2021-12-20'
X[start:end]

,Mon,Tue,Wed,Thu,Fri,Sat,Sun,holiday,daylight_hrs,PRCP,dry day,Temp (C),annual,pandemic
Date,,,,,,,,,,,,,,
2021-12-10,0,0,0,0,1,0,0,0.0,8.304641,3.8,0,4.3,9.191781,1
2021-12-11,0,0,0,0,0,1,0,0.0,8.290187,12.2,0,6.9,9.194521,1
2021-12-12,0,0,0,0,0,0,1,0.0,8.277039,4.3,0,3.9,9.197260,1
2021-12-13,1,0,0,0,0,0,0,0.0,8.265207,3.3,0,3.8,9.200000,1
2021-12-14,0,1,0,0,0,0,0,0.0,8.254701,3.6,0,3.4,9.202740,1
2021-12-15,0,0,1,0,0,0,0,0.0,8.245532,2.0,0,3.7,9.205479,1
2021-12-16,0,0,0,1,0,0,0,0.0,8.237706,1.3,0,4.0,9.208219,1
2021-12-17,0,0,0,0,1,0,0,0.0,8.231231,4.6,0,4.1,9.210959,1
2021-12-18,0,0,0,0,0,1,0,0.0,8.226113,2.3,0,5.3,9.213699,1


Aha, hier ist in den Wetterdaten ``PRCP`` am 18.12.2021 offenbar ein Fehler. Da das Wetter in gemäßigten Breiten (wie Seattle) doch einigermaßen stetig ist, füllen wir diese Lücke durch den Mittelwert des Niederschlags am Vortag und Folgetag:

In [124]:
X.loc['2021-12-18','PRCP'] = 0.5*(X.loc['2021-12-17','PRCP'] + X.loc['2021-12-19','PRCP'])

c:\Users\Cris\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Damit uns das nicht gleich nochmal passiert, prüfen wir, ob sonst noch Lücken in den Daten sind:

In der Tat... Der Dezember 2021 war wohl keine gute Zeit für den Regensensor am Tacoma Airport... Auch diese Lücken füllen wir analog zu oben:

Nun versuchen wir nochmal, das Modell zu trainieren und dann den oben beschriebenen Plots zu erstellen:

Betrachten Sie auch kleinere Zeiträume im Plot (im DataFrame ``df`` können durch ``df.loc['Startdatum':'Enddatum']`` nur die entsprechenden Daten gefiltert werden).

Da wir ein lineares Modell verwendet haben, können wir uns die einzelnen Koeffizienten gut vorstellen; diese geben jeweils an, wie stark das entsprechende Feature in den vorhergesagten Wert eingeht. Die Koeffizienten sind nach dem Training in ``model.coef_`` verfügbar und beziehen sich der Reihe nach auf die Features. Hier eine übersichtliche tabellarische Darstellung:

- Interpretieren Sie diese Daten.
- Um die Güte dieser Abhängigkeiten einschätzen zu können, brauchen wir die Standardabweichungen dieser Werte. Diese erhalten wir, indem wir das Modell mehrfach (z.B. 1000 Mal) auf zufällig ausgewählten Daten trainieren. Dadurch erhalten wir für jeden Koeffizienten 1000 Werte, davon bestimmen wir die Standardabweichung.

In [134]:
from sklearn.utils import resample
np.random.seed(1)
err = np.std([model.fit(*resample(X, y)).coef_
              for i in range(1000)], 0)

In [44]:
print(pd.DataFrame({'effect': params.round(0),
                    'error': err.round(0)}))

              effect  error
Mon            493.0   64.0
Tue            602.0   65.0
Wed            601.0   64.0
Thu            475.0   63.0
Fri            184.0   62.0
Sat           -949.0   66.0
Sun          -1049.0   65.0
holiday      -1101.0   91.0
daylight_hrs   114.0    7.0
PRCP           -27.0    2.0
dry day        532.0   26.0
Temp (C)        70.0    3.0
annual          84.0    6.0
pandemic     -1212.0   47.0


Interpretieren Sie diese Daten.